In [1]:
import os
import getpass

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain_chroma import Chroma

from datetime import datetime, timedelta
import requests

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GENERATIVE_LANGUAGE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
WEATHER_API = os.getenv('WEATHER_API')

In [3]:
def get_date_for_day(day_name):
    """
    Função para construir a data para um dia da semana específico.
    """
    today = datetime.today()
    today_weekday = today.weekday()

    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    try:
        requested_weekday = days_of_week.index(day_name)
    except ValueError:
        return None

    if requested_weekday == today_weekday:
       return today.strftime("%Y-%m-%d")
    
    days_until_requested = (requested_weekday - today_weekday + 7) % 7
    target_date = today + timedelta(days=days_until_requested)
    print(target_date)
    return target_date.strftime("%Y-%m-%d")

In [4]:
import requests

BASE_URL = "http://api.weatherapi.com/v1/forecast.json"

def weatherapi_forecast_periods(date_string: str) -> str:
    """
    Obtém a previsão do tempo para a cidade de Natal em uma data específica,
    separada em manhã, tarde e noite.

    Args:
        date_string (str): Uma string contendo a data no formato yyyy-mm-dd.

    Returns:
        str: Uma string contendo as previsões separadas por períodos para a data especificada.
    """
    try:
        params = {
            "key": WEATHER_API,
            "q": "Natal",
            "dt": date_string,
            "aqi": "no",
            "alerts": "no",
            "lang": "pt"
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if data and data.get("forecast") and data["forecast"].get("forecastday"):
            forecast_data = data["forecast"]["forecastday"][0]
            hourly_data = forecast_data.get("hour", [])

            periods = {
                "Manhã": range(5, 12),
                "Tarde": range(13, 18),
                "Noite": range(18, 24)
            }

            result = f"Previsão para {date_string} em Natal:\n"
            for period, hours in periods.items():
                result += f"\n{period}:\n"
                filtered_data = [
                    {
                        "hora": hour["time"].split(" ")[1],
                        "temperatura": hour["temp_c"],
                        "condicao": hour["condition"]["text"],
                        "chance_de_chuva": hour["chance_of_rain"],
                        "umidade": hour["humidity"]
                    }
                    for hour in hourly_data if int(hour["time"].split(" ")[1].split(":")[0]) in hours
                ]
                if filtered_data:
                    for hour in filtered_data:
                        result += (
                            f"  Hora: {hour['hora']} - "
                            f"Temp: {hour['temperatura']}°C, "
                            f"Condição: {hour['condicao']}, "
                            f"Chance de chuva: {hour['chance_de_chuva']}%, "
                            f"Umidade: {hour['umidade']}%\n"
                        )
                else:
                    result += "  Nenhuma previsão encontrada para este período.\n"

            return result
        else:
            return "Previsão não encontrada para essa data."
    except requests.exceptions.RequestException as e:
        return f"Erro ao buscar informações meteorológicas: {str(e)}"
    except Exception as e:
        return f"Erro inesperado: {str(e)}"


In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [6]:
from get_embedding_function import get_embedding_function
CHROMA_PATH = "chroma"

def query_rag(query_text):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text

def query_rag_tool(query_text: str) -> str:
    return query_rag(query_text)

In [7]:
ddg_search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta deve ser utilizada quando o modelo precisar buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta deve ser usada apenas quando o modelo já tiver todas as informações necessárias para gerar o roteiro e o usuário já tiver fornecido as datas exatas. 
        Se o usuário pedir informações para um período de tempo, o modelo deve consultar o clima separadamente para cada dia desse período. 
        Ela deve ser usada quando a previsão do tempo for um fator importante para o planejamento, mas apenas depois que o modelo tiver claro os detalhes do itinerário."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag_tool,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    )
]

In [8]:
agent_prompt = hub.pull("tales/agente_turismo")

In [9]:
print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'data_atual', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_turismo', 'lc_hub_commit_hash': '42b09d3200bc373c73a85645c84950cda85c7c7fc10222d32ec60815b26f7457'} template='Seu nome é Sofia. Você é uma assistente pessoal de inteligência artificial de classe mundial, especializada em viagens e turismo. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações, planejar roteiros personalizados e resolver problemas relacionados a viagens. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática) com foco em planejamento de viagens.\n\n"Sofia" foi projetada para auxiliar usuários em todas as etapas de suas aventuras, desde a escolha do destino até a criação de itinerários detalhados e dicas práticas. Como modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas informações fornecidas pelos usuá

In [10]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [11]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_28332\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [12]:
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [13]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [14]:
print(data_atual)

sábado, 2024/12/28


In [15]:
response = agent_executor.invoke({"input": f"""
                                  Boa noite
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Boa noite! Para onde você está planejando viajar?
```


> Finished chain.
Boa noite! Para onde você está planejando viajar?
```


In [16]:
response = agent_executor.invoke({"input": f"""
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma bela escolha. Quantos dias você pretende passar lá?
```


> Finished chain.
Ótimo! Natal é uma bela escolha. Quantos dias você pretende passar lá?
```


In [17]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo passar da segunda dia 30 até a quarta
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! De segunda-feira, dia 30, até quarta-feira, dia 01. Isso significa 3 dias em Natal. Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


> Finished chain.
Perfeito! De segunda-feira, dia 30, até quarta-feira, dia 01. Isso significa 3 dias em Natal. Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?
```


In [18]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo visitar praias e restaurantes regionais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores restaurantes regionais em Natal e praias em NatalPRAIA DA REDINHA
A última praia ao norte de Natal é uma antiga vila de pescadores, cujo destaque é o 
Mercado Público. A tradição ali é comer a ginga com tapioca (peixe frito que vem no 
palito, dentro de uma tapioca). A Igreja de Nossa Senhora dos Navegantes, construída 
com pedras dos arrecifes, e o Aquário Natal são outros pontos de interesse.
PRAIA DO MEIO
Situada entre a Praia do Forte 
e a Praia dos Artistas, sedia 
anualmente, durante o Réveillon, 
a tradicional festa de Iemanjá. Era 
considerada a praia mais turística 
da cidade até os anos 1980, 
devido às formações de piscinas 
naturais da região.
PRAIA DOS ARTISTAS
A 5 km do centro de Natal, é uma  
das mais agitadas da cidade. Suas 
ondas altas são ótimas para surfe 
e outros esportes náuticos. Com

---

PRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 
hoteleira, 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Praias e restaurantes regionais são escolhas excelentes em Natal.  Para finalizar o planejamento, preciso saber qual o seu orçamento para a viagem. Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?
```


> Finished chain.
Ótimo! Praias e restaurantes regionais são escolhas excelentes em Natal.  Para finalizar o planejamento, preciso saber qual o seu orçamento para a viagem. Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?
```


In [19]:
response = agent_executor.invoke({"input": f"""
                                  Sem incluir Hóteis pretendo gastar no máximo 600 reais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Certo! R$600 para três dias em Natal, focado em praias e restaurantes regionais.  Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


> Finished chain.
Certo! R$600 para três dias em Natal, focado em praias e restaurantes regionais.  Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


In [20]:
response = agent_executor.invoke({"input": f"""
                                  Eu não possuo nenhuma alergia
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! Sem alergias.  Há mais algum detalhe importante sobre sua viagem que você gostaria de compartilhar? Por exemplo, prefere viagens relaxantes ou mais dinâmicas?
```


> Finished chain.
Perfeito! Sem alergias.  Há mais algum detalhe importante sobre sua viagem que você gostaria de compartilhar? Por exemplo, prefere viagens relaxantes ou mais dinâmicas?
```


In [21]:
response = agent_executor.invoke({"input": f"""
                                  Não.
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores restaurantes regionais em Natal, Rio Grande do Norte e melhores praias em NatalPRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 
hoteleira, com quiosques atendendo à beira-mar. Da movimentada 
Ponta Negra à tradicional Redinha, tem praia para todos os gostos!
VIA COSTEIRA
A continuação da Praia de Ponta Negra acompanha a famosa Via Costeira, principal 
avenida da cidade. Nessa faixa de areia, ficam os incríveis resorts da cidade, com 
estrutura completa de lazer e gastronomia. A bela paisagem da região é pontuada por 
dunas, trechos de Mata Atlântica e também lindas praias desertas.
PONTA NEGRA
O Morro do Careca é o marco principal da Praia de Ponta Negra e cartão-postal de 
Natal. Emoldurada por vegetação, a duna de 107 m de altura domina a paisagem – 
mas é proibido subir nela por questões ambientais. A praia em si é supermovimentada,

---

PRAIA DA REDINHA
A última praia 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30, 2024-12-31, 2025-01-01Previsão para 2024-12-30, 2024-12-31, 2025-01-01 em Natal:

Manhã:
  Hora: 05:00 - Temp: 25.9°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 76%, Umidade: 81%
  Hora: 06:00 - Temp: 27.2°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 75%
  Hora: 07:00 - Temp: 28.2°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 71%
  Hora: 08:00 - Temp: 28.9°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 68%
  Hora: 09:00 - Temp: 29.7°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 70%, Umidade: 64%
  Hora: 10:00 - Temp: 30.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 62%
  Hora: 11:00 - Temp: 30.1°C, Condição: Parcialmente nublado, Chance de chuva: 89%, Umidade: 66%

Tarde:
  Hora: 13:00 - Temp: 29.7°C, Condição: Possibilidade de chuv

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com base nas suas preferências e no seu orçamento, preparei um roteiro para sua viagem a Natal.  Considerando que você estará na cidade de segunda (30/12) a quarta-feira (01/01), e levando em conta a previsão do tempo, que indica possibilidade de chuva em alguns momentos, vamos priorizar atividades que possam ser adaptadas caso chova.  Para os restaurantes, selecionei opções com preços que se encaixam no seu orçamento (R$600 para 3 dias, excluindo hospedagem). Lembre-se que esses são apenas sugestões e você pode adaptá-las conforme sua preferência.  Gostaria de ver o roteiro?
```


> Finished chain.
Ótimo! Com base nas suas preferências e no seu orçamento, preparei um roteiro para sua viagem a Natal.  Considerando que você estará na cidade de segunda (30/12) a quarta-feira (01/01), e levando em conta a previsão do tempo, que indica possibilidade de chuva em alguns momentos, vamos priorizar atividades que possam ser 

In [22]:
response = agent_executor.invoke({"input": f"""
                                  Sim.
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024-12-30, 2024-12-31, 2025-01-01Previsão para 2024-12-30, 2024-12-31, 2025-01-01 em Natal:

Manhã:
  Hora: 05:00 - Temp: 25.9°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 76%, Umidade: 81%
  Hora: 06:00 - Temp: 27.2°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 75%
  Hora: 07:00 - Temp: 28.2°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 71%
  Hora: 08:00 - Temp: 28.9°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 68%
  Hora: 09:00 - Temp: 29.7°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 70%, Umidade: 64%
  Hora: 10:00 - Temp: 30.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 62%
  Hora: 11:00 - Temp: 30.1°C, Condição: Parcialmente nublado, Chance de chuva: 89%, Umidade: 66%

Tarde:
  Hora: 13:00 - Temp: 29.7°C, Condição: Possibilidade de chuv

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: pontos turísticos e restaurantes em Natal, Rio Grande do Norte, focados em praias e culinária regionalPRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 
hoteleira, com quiosques atendendo à beira-mar. Da movimentada 
Ponta Negra à tradicional Redinha, tem praia para todos os gostos!
VIA COSTEIRA
A continuação da Praia de Ponta Negra acompanha a famosa Via Costeira, principal 
avenida da cidade. Nessa faixa de areia, ficam os incríveis resorts da cidade, com 
estrutura completa de lazer e gastronomia. A bela paisagem da região é pontuada por 
dunas, trechos de Mata Atlântica e também lindas praias desertas.
PONTA NEGRA
O Morro do Careca é o marco principal da Praia de Ponta Negra e cartão-postal de 
Natal. Emoldurada por vegetação, a duna de 107 m de altura domina a paisagem – 
mas é proibido subir nela por questões ambientais. A praia em si é supermovimentada,

---

PRAIA DA REDINHA
A

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Roteiro de viagem para Natal (RN) - 3 dias (30/12 a 01/01)

**Considerações:** Este roteiro leva em conta a previsão do tempo (possibilidade de chuva), seu orçamento (R$600, excluindo hospedagem) e o seu interesse em praias e restaurantes regionais.  As sugestões de restaurantes incluem opções com preços acessíveis.  Adapte o roteiro à sua conveniência.

**Segunda-feira (30/12):**

* **Manhã (8h-12h):**  Devido à alta probabilidade de chuva pela manhã, sugiro começar o dia visitando o Aquário de Natal, na Praia da Redinha.  É uma atividade coberta e interessante.  Após a visita, aproveite para almoçar em um restaurante próximo à praia, experimentando a culinária regional.  Sugestão:  Restaurante com frutos do mar na Redinha (pesquise opções locais com preços acessíveis).
* **Tarde (14h-18h):**  Se a chuva cessar, aproveite a tarde na Praia de Ponta Negra.  Caso contrário, visite o Centro de Turismo, que abriga lojas de art

In [ ]:
print(response["chat_history"])